A colab file for finding the number of time our knees gets bend using mediapipe

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.0 MB 1.3 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 22.11.23 which is incompatible.


In [2]:
import cv2
import time
import math as m
import mediapipe as mp
import numpy as np

# Function for finding angle

For finding the angle we will be using the vector distance approach betwene two co-ordinates 

In [3]:
def findangle(p0,p1,p2):
  v0 = np.array(p0) - np.array(p1)
  v1 = np.array(p2) - np.array(p1)

  return np.degrees(np.math.atan2(np.linalg.det([v0, v1]), np.dot(v0, v1)))

# Function for sending Warning 

In [4]:
def sendWarning():
  print("Keep your knee bent")
  pass

# Initializations

In [5]:
# Initialize frame counters.
good_frames = 0
bad_frames  = 0

In [6]:
# Font type.
font = cv2.FONT_HERSHEY_SIMPLEX

In [7]:
# Colors.
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)

In [8]:
# Initialize mediapipe pose class.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Video capture and writer 

In [9]:
file_name = '/content/drive/MyDrive/datasets/KneeBendVideo.mp4'
cap = cv2.VideoCapture(file_name)

In [10]:
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

In [11]:
# Video writer.
video_output = cv2.VideoWriter('output.mp4', fourcc, fps, frame_size)

# Capturing the pose

In [12]:
# Capture frames.
success, image = cap.read()

In [13]:
# Get fps.
fps = cap.get(cv2.CAP_PROP_FPS)

In [14]:
# Get height and width of the frame.
h, w = image.shape[:2]

In [15]:
# Convert the BGR image to RGB.
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [16]:
# Process the image.
keypoints = pose.process(image)

In [17]:
# Convert the image back to BGR.
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [18]:
# Use lm and lmPose as representative of the following methods.
lm = keypoints.pose_landmarks
lmPose  = mp_pose.PoseLandmark

In [19]:
#hip co-ordiantes
# right hip.
r_hip_x = int(lm.landmark[lmPose.RIGHT_HIP].x * w)
r_hip_y = int(lm.landmark[lmPose.RIGHT_HIP].y * h)

In [20]:
#knee co-ordinates 
r_knee_x = int(lm.landmark[lmPose.RIGHT_KNEE].x * w)
r_knee_y = int(lm.landmark[lmPose.RIGHT_KNEE].y * h)

In [21]:
#right ankle co-ordinates 
r_ankle_x = int(lm.landmark[lmPose.RIGHT_ANKLE].x * w)
r_ankle_y = int(lm.landmark[lmPose.RIGHT_ANKLE].y * h)

# Calculating the incliantion

# Counting the REp for knee

In [23]:
print('Processing..')
while cap.isOpened():
    # Capture frames.
    success, image = cap.read()
    if not success:
        print("Null.Frames")
        break
    # Get fps.
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Get height and width.
    h, w = image.shape[:2]

    # Convert the BGR image to RGB.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image.
    keypoints = pose.process(image)

    # Convert the image back to BGR.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Use lm and lmPose as representative of the following methods.
    lm = keypoints.pose_landmarks
    lmPose = mp_pose.PoseLandmark

    # Acquire the landmark coordinates.
    # Once aligned properly, left or right should not be a concern.      
    # Right HIP
    if lm is not None:
      r_hip_x = int(lm.landmark[lmPose.RIGHT_HIP].x * w)
      r_hip_y = int(lm.landmark[lmPose.RIGHT_HIP].y * h)
      # Right knee
      r_knee_x = int(lm.landmark[lmPose.RIGHT_KNEE].x * w)
      r_knee_y = int(lm.landmark[lmPose.RIGHT_KNEE].y * h)
      # Right Ankle.
      r_ankle_x = int(lm.landmark[lmPose.RIGHT_ANKLE].x * w)
      r_ankle_y = int(lm.landmark[lmPose.RIGHT_ANKLE].y * h)    


    angle= -(findangle([r_hip_x,1-r_hip_y],[r_knee_x,1-r_knee_y],[r_ankle_x,1-r_ankle_y]))

    # Put text, Posture and angle inclination.
    # Text string for display.
    angle_text_string = (f'The angle of knee bending is {angle}')

    # Determine whether good posture or bad posture.
    # The threshold angles have been set based on intuition.
    if (angle<140):
      bad_frames = 0
      good_frames += 1
      good_time = (1 / fps) * good_frames
      bad_time =  (1 / fps) * bad_frames
      #putting angle on each part
      cv2.putText(image, angle_text_string, (10, 30), font, 0.9, light_green, 2)
      # Join landmarks.
      cv2.line(image, (r_hip_x, r_hip_y), (r_knee_x, r_knee_y), green, 4)
      cv2.line(image, (r_knee_x, r_knee_y), (r_ankle_x, r_ankle_y), green, 4)
      time_string_good = 'Good Posture Time : ' + str(round(good_time, 1)) + 's'
      cv2.putText(image, time_string_good, (10, h - 20), font, 0.9, green, 2)

      if good_time>8: 
        cv2.putText(image,'Now pls rest you knee',(50,50),font,0.9,red,2)

      else: 
        cv2.putText(image,'Keep your knee bend ',(50,50),font,0.9,red,2)

      

    else: 
      good_frames = 0
      bad_frames += 1
      good_time = (1 / fps) * good_frames
      bad_time =  (1 / fps) * bad_frames
      #putting angle on each part
      cv2.putText(image, angle_text_string, (10, 30), font, 0.9, light_green, 2)
      
      # Join landmarks.
      cv2.line(image, (r_hip_x, r_hip_y), (r_knee_x, r_knee_y), green, 4)
      cv2.line(image, (r_knee_x, r_knee_y), (r_ankle_x, r_ankle_y), green, 4)
      time_string_bad = 'Bad Posture Time : ' + str(round(bad_time, 1)) + 's'
      cv2.putText(image, time_string_bad, (10, h - 20), font, 0.9, red, 2)

      cv2.putText(image,'Please bend your knee',(50,50),font,0.9,red,2)

    # Write frames.
    video_output.write(image)
print('Finished.')
cap.release()
video_output.release()

Processing..
Null.Frames
Finished.
